#### Setting up game configurations

In [1]:
from vizdoom import *
import random # for random actions
import time
import numpy as np

# Open AI Gym dependencies
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box # for random actions and nxm for random observation space (frames)
import cv2

# Stable Baselines3 dependencies
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# Optimizer
import optuna 

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/acaia/VizDoom_TakeCover_RL/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Converting it to a Gym Environment

In [2]:
# Game configuration : Take Cover configuration file
config = 'github_vizdoom_repo/ViZDoom/scenarios/take_cover_lev1.cfg'

In [3]:
# Creating Vizdoom OpenAI Gym Environment
class VizDoomGym(Env):
    def __init__(self, render_mode = False): # By default, rendering is disabled
        # Inheriting from the Env class
        super().__init__()
        # Setup game
        self.game = DoomGame()
        self.game.load_config(config) 

        # Rendering mode : if unabled, the game will not be displayed but the training will be faster
        if render_mode == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        # Start the game
        self.game.init()
        
        # In order to get the game frame size, run a dummy demo and get the screen buffer shape  with game.get_state().screen_buffer.shape
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype = np.uint8)
        # Action space
        self.action_space = Discrete(2) # left, right
        self.current_step = 0


    # Defining how to make a step in the env
    def step(self, action):
        actions = np.identity(2, dtype=np.uint8) # Possible actions [left, right]
        reward = self.game.make_action(actions[action], 4) # Defyining the frame skip parameter to 4

        # Check if the frames are over
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            obs = self.grayscale(state)
            health = self.game.get_state().game_variables[0]
            terminated = self.game.is_episode_finished()
            truncated = self.current_step >= 2100  # Max steps
            info = {"health": health}
        else: # Default zeros observation
            obs = np.zeros(self.observation_space.shape, dtype=np.uint8)
            terminated = True
            truncated = False
            info = {} # Empty info: no 0 beacuse the API doesn't allow it

        self.current_step += 1
        return obs, reward, terminated, truncated, info # Changed parameters order according to Gymnasium API
    
    def render():
        pass
    
    # What appens when starting a new episode
    def reset(self, seed = None, options = None):
        super().reset(seed=seed)
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        obs = self.grayscale(state)
        info = {"health": self.game.get_state().game_variables[0]}
        self.current_step = 0

        return obs, info

    # Grayscale and resize the frames in order to reduce the observation space
    ## POSSIBLE IMPROVEMENT : CUT OFF BOTTOM PART OF THE IMAGE WHERE THERE IS NO USEFUL INFORMATION
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY) # moveaxis moves the first element (0) to last position (-1)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160, 1))
        return state

    def close(self):
        self.game.close()

In [4]:
# Check pass to see if the environment works
env_checker.check_env(env=VizDoomGym())

### Setting up Callbacks

In [5]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}.zip'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [6]:
CHECKPOINT_DIR = './train/train_DeadlyCorridor'
OPT_DIR = './opt/opt_DeadlyCorridor'
LOG_DIR = './logs/log_DeadlyCorridor'

In [7]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path = CHECKPOINT_DIR)

### Hyperparameters Tuning with Optuna

In [ ]:
# Objective function to test hyperparameters
def optimize_ppo(trial):
    return {
        'n_steps': trial.suggest_int('n_steps', 2048, 8192, step = 64), # number of frames used in a single training step (must be multiple of 64)
        'gamma': trial.log_uniform('gamma', 0.8, 0.9999), # discount factor
        'learning_rate': trial.log_uniform('learning_rate', 1e-5, 1e-4),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [13]:
# Run a training loop and return the mean reward
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) # set of hyperparameters to test

        # Create new env
        env = VizDoomGym(config=config)
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Set up, train and evaluatethe PPO model
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=100000)
        
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        # Save the best model 
        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward


    except Exception as e:
        print(e)
        return -1000 # Return a very low reward if training fails

In [14]:
# Begin an optuna study
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2026-01-02 14:29:59,631] A new study created in memory with name: no-name-b8c6d571-8ac4-4ba1-bbcd-e44738d5e433
[I 2026-01-02 14:29:59,637] Trial 0 finished with value: -1000.0 and parameters: {'n_steps': 5440}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,639] Trial 1 finished with value: -1000.0 and parameters: {'n_steps': 6336}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,640] Trial 2 finished with value: -1000.0 and parameters: {'n_steps': 7168}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,642] Trial 3 finished with value: -1000.0 and parameters: {'n_steps': 5376}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,643] Trial 4 finished with value: -1000.0 and parameters: {'n_steps': 6400}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,646] Trial 5 finished with value: -1000.0 and parameters: {'n_steps': 7168}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,648] Trial 6 finished with value: -1000.0

'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attrib

[I 2026-01-02 14:29:59,843] Trial 50 finished with value: -1000.0 and parameters: {'n_steps': 7360}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,848] Trial 51 finished with value: -1000.0 and parameters: {'n_steps': 6912}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,853] Trial 52 finished with value: -1000.0 and parameters: {'n_steps': 7872}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,857] Trial 53 finished with value: -1000.0 and parameters: {'n_steps': 7552}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,863] Trial 54 finished with value: -1000.0 and parameters: {'n_steps': 7104}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,869] Trial 55 finished with value: -1000.0 and parameters: {'n_steps': 6464}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,876] Trial 56 finished with value: -1000.0 and parameters: {'n_steps': 4672}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:29:59,891] Trial 

'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attrib

[I 2026-01-02 14:30:00,045] Trial 85 finished with value: -1000.0 and parameters: {'n_steps': 7232}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,049] Trial 86 finished with value: -1000.0 and parameters: {'n_steps': 6656}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,055] Trial 87 finished with value: -1000.0 and parameters: {'n_steps': 7488}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,060] Trial 88 finished with value: -1000.0 and parameters: {'n_steps': 3264}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,066] Trial 89 finished with value: -1000.0 and parameters: {'n_steps': 6272}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,072] Trial 90 finished with value: -1000.0 and parameters: {'n_steps': 8000}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,078] Trial 91 finished with value: -1000.0 and parameters: {'n_steps': 6528}. Best is trial 0 with value: -1000.0.
[I 2026-01-02 14:30:00,086] Trial 

'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'
'Trial' object has no attribute 'suggest_longuniform'


In [12]:
# Print the best hyperparameters
print("Best hyperparameters combo: ", study.best_params)
print("Best trial: ", study.best_trial)

Best hyperparameters combo:  {'n_steps': 4544}
Best trial:  FrozenTrial(number=0, state=<TrialState.COMPLETE: 1>, values=[-1000.0], datetime_start=datetime.datetime(2026, 1, 2, 14, 28, 41, 769227), datetime_complete=datetime.datetime(2026, 1, 2, 14, 28, 41, 770548), params={'n_steps': 4544}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=64)}, trial_id=0, value=None)


### Fine-Tuning the best Agent

In [ ]:
env = VizDoomGym(config=config)

In [ ]:
# instantiate PPO model
model = PPO('CnnPolicy', # policy type -> CnnPolicy since we are working on image frames
            env,
            tensorboard_log=LOG_DIR,
            verbose=1,
            learning_rate=study.best_params['learning_rate'],
            n_steps=study.best_params['n_steps'], 
            clip_range=study.best_params['clip_range'],
            gamma= study.best_params['gamma'],
            gae_lambda=study.best_params['gae_lambda']
)

In [ ]:
model.learn(total_timesteps = 400000, callback=callback)

### Testing the trained agent on real-time game

In [ ]:
# Reload the best model from disk
model = PPO.load("./train/train_basic/best_model_400000.zip")

In [ ]:
env = VizDoomGym(render_mode= True) # rendered-env

In [ ]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)

In [ ]:
env.close()